In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
import seaborn as sns
import json
from io import StringIO
import importlib

In [ ]:
import molten_data_common_lib
importlib.reload(molten_data_common_lib)
from molten_data_common_lib import glob_file_list , load_json_from_file, merge_dicts, plot_groups, get_varying_column_names

In [ ]:
src_files = []
#src_files += ['../driver-pravega/logs/workload-01c9e1d5-ef0e-4f73-9c1e-d6f7e3023aef-Pravega-2019-09-20-01-23-42.json']
#src_files += ['../driver-pravega/logs/workload-07098146-76a8-40ad-9377-c4a7964a96b4-Pravega-2019-09-19-22-52-16.json']
src_files += ['../driver-pravega/logs/workload-*.json']
filenames = glob_file_list(src_files)
print('Loading records from %d files...' % len(filenames))
raw_results = [load_json_from_file(filename) for filename in filenames]

In [ ]:
raw_df = pd.DataFrame(raw_results)
raw_df.head()

In [ ]:
def clean_result(result):
    r = result.copy()
    workload = json.load(StringIO(r['workload']))
    r = merge_dicts(r, workload)
    r = merge_dicts(r, r['workload'])
    del r['workload']
    r['driverName'] = r['driver']['name']
    del r['driver']
    for k in list(r.keys()):
        if 'Quantiles' in k:
            r[k] = pd.Series(data=[float(q) for q in r[k].keys()], index=list(r[k].values()))
        elif isinstance(r[k], list):
            r[k] = pd.Series(r[k])
            r['%sMean' % k] = r[k].mean()
    return pd.Series(r)
# r = clean_result(raw_results[0])
# pd.DataFrame(r)

In [ ]:
clean_df = raw_df.apply(clean_result, axis=1)
clean_df = clean_df.sort_values(['utc_begin'])
# clean_df.tail()

In [ ]:
clean_df.columns

In [ ]:
clean_df.tail(20)[['utc_begin','publishRateMean','aggregatedPublishLatencyAvg','aggregatedPublishLatency99pct']]

In [ ]:
df = clean_df[
    (clean_df.messageSize==10000) & 
    (clean_df.testDurationMinutes==5) & 
    (clean_df.messageSize==10000) & 
    (clean_df.partitionsPerTopic==1) &
    (clean_df.producerRate==10)
]
#df = df[['utc_begin','producerRate','partitionsPerTopic','messageSize','publishRateMean','aggregatedPublishLatency99pct']]
#get_varying_column_names(df)

In [ ]:
plot_groups(
    clean_df[(clean_df.messageSize==10000) & (clean_df.testDurationMinutes==5)], 
    x_col='publishRateMean', 
    y_col='aggregatedPublishLatency99pct',
    group_by_columns=['partitionsPerTopic', 'messageSize'],
    semilogx=True,
    ylim=[0,100],
);